In [3]:
import MetaTrader5 as mt5
import time
import math
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

from forex_python.converter import CurrencyRates
from currency_converter import CurrencyConverter

from random import sample

from tqdm import tqdm
import vectorbt as vbt

from pandas.tseries.offsets import BDay
from itertools import combinations, count

import statsmodels.api as sm
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.tsa.stattools import adfuller

from threading import Thread, Lock
from multiprocessing import Process

import requests
import json
import hashlib

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cycler

In [4]:
colors = cycler('color', ['669FEE', '66EE91', '9988DD', 'EECC55', '88BB44', 'FFBBBB'])
plt.rc('figure', facecolor='313233')
plt.rc('axes', facecolor='313233', edgecolor='none', axisbelow=True, grid=True, prop_cycle=colors, labelcolor='gray')
plt.rc('grid', color='474A4A', linestyle='solid')
plt.rc('xtick', color='gray')
plt.rc('ytick', direction='out', color='gray')
plt.rc('legend', facecolor='313233', edgecolor='313233')
plt.rc('text')

In [5]:
def send_telegram_message(message: str, chat_id: str, api_key: str, ):
    responses = {}

    proxies = None
    headers = {'Content-Type': 'application/json',
               'Proxy-Authorization': 'Basic base64'}
    data_dict = {'chat_id': chat_id,
                 'text': message,
                 'parse_mode': 'HTML',
                 'disable_notification': True}
    data = json.dumps(data_dict)
    url = f'https://api.telegram.org/bot{api_key}/sendMessage'

    requests.packages.urllib3.disable_warnings()
    response = requests.post(url,
                             data=data,
                             headers=headers,
                             proxies=proxies,
                             verify=False)
    return response

In [6]:
def get_symbols(location):
    mt5.initialize()

    symbol_list = []
    for symbol in mt5.symbols_get():
        if location in symbol.path:
            symbol_list.append(symbol.name)

    return symbol_list

In [7]:
def get_bars_from_broker(symbol, start_date, end_date, timeframe=mt5.TIMEFRAME_M1):
    mt5.initialize()
    bars = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    df_bars = pd.DataFrame(bars)
    df_bars["time"] = pd.to_datetime(df_bars["time"], unit="s")
    return df_bars

In [8]:
def get_latest_bar(symbol, timeframe=mt5.TIMEFRAME_M1):
    mt5.initialize()
    bar = mt5.copy_rates_from(
        symbol,
        timeframe,
        datetime.now() + timedelta(hours=3),
        1)  # number of bars
    df_bar = pd.DataFrame(bar)
    df_bar["time"] = pd.to_datetime(df_bar["time"], unit="s")
    return df_bar

In [9]:
def new_bar_event_pairs(name_1, name_2, last_bar_1, last_bar_2):
    mt5.initialize()

    while True:

        new_bar_1 = get_latest_bar(name_1)
        new_bar_2 = get_latest_bar(name_2)

        if new_bar_1['time'].iloc[0] != last_bar_1['time'].iloc[0] or new_bar_2['time'].iloc[0] != \
                last_bar_2['time'].iloc[0]:
            print(f'{name_1}, {name_2}: new tick: {datetime.now()}')
            return

        time.sleep(5)

In [10]:
def get_bars(symbol):
    return pd.read_csv(symbol + '.csv')

In [11]:
def find_filling_mode(symbol):
    mt5.initialize()

    for i in range(2):
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": mt5.symbol_info(symbol).volume_min,
            "type": mt5.ORDER_TYPE_BUY,
            "price": mt5.symbol_info_tick(symbol).ask,
            "type_filling": i,
            "type_time": mt5.ORDER_TIME_GTC}

        result = mt5.order_check(request)

        if result.comment == "Done":
            break

    return i